In [1]:
import requests as rq
import json
import pandas as pd

In [2]:
# Get API key
def get_key():
    f = open("/home/vikas/Documents/CG_key.json")
    key_dict = json.load(f)
    return key_dict["key"] 

## Check API status

In [3]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

In [4]:
def get_response(endpoint, headers, params, URL):
    url = "".join((URL, endpoint))
    response = rq.get(url, headers = headers, params = params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data, check status code {response.status_code}")    

In [5]:
use_demo = {"accept": "application/json"}

In [6]:
use_pro = {
         "accept": "application/json",
         "x-cg-pro-api-key" : get_key()          
}

In [10]:
get_response("/ping", use_demo, "", PUB_URL)

{'gecko_says': '(V3) To the Moon!'}

## NFTs

#### Demo API

In [11]:
nft_params = {
            "order": "floor_price_native_desc",
            "per_page": 10,
            "page": 1
}

In [12]:
nft_list_response = get_response("/nfts/list", use_demo, nft_params, PUB_URL)

In [13]:
nft_list_response;

In [14]:
df_nft = pd.DataFrame(nft_list_response)

In [15]:
df_nft

,id,contract_address,name,asset_platform_id,symbol
0,gaia-supernova-deprecated,0x89a18abab20aab069feb7cab20517630ee7c1626,Gaia Supernova,klay-token,GAIA
1,gaia-genesis-deprecated,0xbb915237d8b46dcdfe813c914bf98708e0dad84a,Gaia Kronos,klay-token,GAIA
2,burned-202209,0xc6e5ba514dc2ec6a556dbb95d3002cc1070ad7b9,Dicast Hero Badge SuperRare Pass,klay-token,DHBSRP
3,gaia-protocol-genesis-klaytn,0xe9a10bb97ddb4bcd7677393405b4b769273cef3c,Gaia Protocol Genesis,klay-token,GAIA
4,syltare-dawn-of-east-klaytn,0xdf104df1b238dcd1be2d160cea47893e03b3f9b9,SYL - Dawn of East,klay-token,SYL
5,pixels-pets,0xb806028b6ebc35926442770a8a8a7aeab6e2ce5c,Pixels Pets,ronin,PIXELS-PETS
6,footage-by-sportium,0x839e43c5a768890f5d35ecdf0fb0b356529a80b5,FOOTAGE BY SPORTIUM,klay-token,FOOTAGE
7,syltare-frozen,0xe2dd65c215089dda1695ae96cce77f301a1750e5,SYL - Dawn of East,klay-token,SYL
8,the-crop-collective,0xe5863d4a57b1e2d7031f7969fee4f56bb04c0f17,The Crop Collective,ethereum,$TCC
9,claynosaurz-sardinhas,,Claynosaurz: Sardinhas,solana,CLAYSARDINHAS


In [16]:
df_nft.id[4]

'syltare-dawn-of-east-klaytn'

In [17]:
def get_nft_current_data(index, df_nft, use_demo = use_demo, PUB_URL = PUB_URL):
    nft_url = f"/nfts/{df_nft.id[index]}"
    nft_current_response = get_response(nft_url, use_demo, nft_params, PUB_URL)
    
    new_data = dict(id = nft_current_response["id"], 
                    name = nft_current_response["name"],
                    symbol = nft_current_response["symbol"],
                    floor_price_usd = nft_current_response["floor_price"]["usd"])
    
    df_nft_current = pd.DataFrame([new_data])
    return df_nft_current

In [18]:
df_nft_current = get_nft_current_data(5, df_nft)

In [19]:
df_nft_current

,id,name,symbol,floor_price_usd
0,pixels-pets,Pixels Pets,PIXELS-PETS,1416.06


#### Pro API

In [31]:
page_params = {
            "per_page": 100,
            "page": 1,
            "order": "h24_volume_usd_desc"    
}

In [32]:
nft_markets = get_response("/nfts/markets", use_pro, page_params, PRO_URL)

In [43]:
nft_markets;

In [42]:
def get_nft_markets(num_entries, order_by):
    
    page_params = {
                    "per_page": num_entries,
                    "page": 1,
                    "order": order_by    
                  }
    
    nft_markets = get_response("/nfts/markets", use_pro, page_params, PRO_URL)    

    for i in range(len(nft_markets)):
        nft_select = nft_markets[i]
        temp_dict = dict(
                         id = nft_select["id"],
                         asset_platform_id  = nft_select["asset_platform_id"],
                         floor_price_native = nft_select["floor_price"]["native_currency"],
                         floor_price_usd    = nft_select["floor_price"]["usd"],
                         market_cap_usd     = nft_select["market_cap"]["usd"]
                        )
        temp_df = pd.DataFrame([temp_dict])
        if i == 0:
            df_nft_markets = temp_df
        else:
            df_nft_markets = df_nft_markets.append(temp_df)
            
    return df_nft_markets                     


In [44]:
df_nft_markets = get_nft_markets(10, "h24_volume_usd_desc")

AttributeError: 'DataFrame' object has no attribute 'append'